# load required modules

In [ ]:
import sys, os, pygmt
sys.path.insert(0, '/home/581/da1339/AFIM/src/AFIM/src')
from sea_ice_processor import SeaIceProcessor
from sea_ice_plotter   import SeaIcePlotter
import numpy           as np
import pandas          as pd
import xarray          as xr
from datetime          import timedelta, date, datetime

# process for fast/pack ice

In [ ]:
dt0_str  = "1998-08-01"
dtN_str  = "1999-03-31"
sim_name = 'baseline'
SI_proc  = SeaIceProcessor(sim_name            = sim_name, 
                           ice_speed_threshold = 1e-4)
FI_lo_spd = SI_proc.process_window(dt0_str    = dt0_str,
                                    dtN_str    = dtN_str, 
                                    write_zarr = False,
                                    ow_zarrs   = True)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
plt.figure(figsize=(12, 6))
plt.style.use('ggplot') 
plt.plot(FI_lo_spd['t_dim'], FI_lo_spd['FIA']    , label=f'{sim_name}-simulation' , linewidth=2, color='tab:blue')
plt.plot(FI_lo_spd['t_dim'], FI_lo_spd['FIA_OBS'], label='Observed FIA (AF2020db)', linewidth=2, linestyle='--', color='tab:orange')
plt.title("Fast Ice Area with simulation results masked with ice speed of <1 micorn/s", fontsize=16)
plt.xlabel("Date", fontsize=14)
plt.ylabel(f"Fast Ice Area ({FI_lo_spd.FIA.attrs.get('units', 'unknown')})", fontsize=14)
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gcf().autofmt_xdate()
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(loc='upper left', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
SI_proc  = SeaIceProcessor(sim_name            = sim_name, 
                           ice_speed_threshold = 1e-3)
FI_hi_spd = SI_proc.process_window(dt0_str    = dt0_str,
                                    dtN_str    = dtN_str, 
                                    write_zarr = False,
                                    ow_zarrs   = True)

In [ ]:
plt.figure(figsize=(12, 6))
plt.style.use('ggplot') 
plt.plot(FI_hi_spd['t_dim'], FI_hi_spd['FIA']    , label=f'{sim_name}-simulation' , linewidth=2, color='tab:blue')
plt.plot(FI_hi_spd['t_dim'], FI_hi_spd['FIA_OBS'], label='Observed FIA (AF2020db)', linewidth=2, linestyle='--', color='tab:orange')
plt.title("Fast Ice Area with simulation results masked with ice speed of <1 mm/s", fontsize=16)
plt.xlabel("Date", fontsize=14)
plt.ylabel(f"Fast Ice Area ({FI_lo_spd.FIA.attrs.get('units', 'unknown')})", fontsize=14)
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gcf().autofmt_xdate()
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(loc='upper left', fontsize=12)
plt.tight_layout()
plt.show()

# plot 1D analysis

In [ ]:
dt0_str  = "1993-01-01"
dtN_str  = "1993-12-31"
sim_name = 'ktens-max'
plotter  = SeaIcePlotter(sim_name, dt0_str, dtN_str, show_figs=True, ice_type='FI')
plotter.plot_timeseries()

In [ ]:
#plotter.plot_timeseries(var_names=["PI_STRESS"])
plotter.plot_timeseries_compare(
    comparison_name = "tensile-stars",
    sim_names       = ["baseline", "ktens-max", "Pstar-max", "Cstar-min"],
    var_name        = "PIA",
    label_dict      = {"baseline"  : "tensile = 0.2, Pstar = 2.75e4, Cstar = 20",
                       "ktens-max" : "tensile = 0.3, Pstar = 2.75e4, Cstar = 20",
                       "Pstar-max" : "tensile = 0.2, Pstar = 5e4   , Cstar = 20",
                       "Cstar-min" : "tensile = 0.2, Pstar = 2.75e4, Cstar = 10"})

# plot 2D analysis (requires existing FI/PI dataset)

In [ ]:
dt0_str  = "1993-01-01"
dtN_str  = "1999-12-31"
sim_names = ["baseline", "ustar-low", "Cstar-max", "Cstar-min", "Pstar-max", "Pstar-min",
             "ktens-nil", "ktens-min", "ktens-max", "gi-max", "gi-min",
             "Rothrock", "elps-pack", "elps-fast", "elps-max", "elps-def", "elps-mid" ]
for sim_name in sim_names:
    SI_plot = SeaIcePlotter(sim_name, dt0_str, dtN_str, show_figs=False, overwrite=True, single_figure=False)
    SI_plot.prepare_data_for_plotting(var_name='FIP')
    SI_plot.plot_regions()

# generate PBS jobs for analysis

In [ ]:
from pathlib import Path
from datetime import datetime, timedelta

sim_names = [
    "Cstar-max", "Cstar-min", "Pstar-max", "Pstar-min", "Rothrock",# "baseline", "oras-sfc-ctl"
    "elps-def", "elps-fast", "elps-max", "elps-mid", "elps-pack", "gi-max",
    "gi-min", "ktens-max", "ktens-min", "ktens-nil", "ustar-low", "van_achter"
]

# Base time: 5 minutes from now
base_time = datetime.now() + timedelta(minutes=5)

pbs_template = """#!/bin/bash
#PBS -N FI-proc-{name}
#PBS -P gv90
#PBS -l walltime=24:00:00
#PBS -q normalbw
#PBS -l mem=32GB
#PBS -l ncpus=28
#PBS -l storage=gdata/cj50+gdata/jk72+scratch/jk72+gdata/ik11+gdata/hh5+gdata/rt52+gdata/gb6+gdata/gv90
#PBS -M daniel.atwater@utas.edu.au
#PBS -a {start_time}

module purge
module use /g/data/hh5/public/modules
module load conda/analysis3

cd ~/AFIM/src/AFIM/scripts
python3 ./compute_sea_ice.py {name} --overwrite
"""

output_dir = Path("/home/581/da1339/AFIM/src/sh/analysis/fast_ice")
output_dir.mkdir(exist_ok=True)

for i, sim in enumerate(sim_names):
    start_dt = base_time + timedelta(minutes=5 * i)
    start_str = start_dt.strftime("%y%m%d%H%M")  # YYMMDDhhmm format
    script_content = pbs_template.format(name=sim, start_time=start_str)
    script_path = output_dir / f"FI-proc_{sim}.pbs"
    with open(script_path, "w") as f:
        f.write(script_content)

print(f"✓ Generated {len(sim_names)} staggered PBS scripts in: {output_dir.resolve()}")
